# Mixed Integer Differential Predictive Control Example


## Problem Formulation

\begin{equation}
x_{k+1} = A x_k + B_u u_k + B_\delta \delta_k + E d_k,
\end{equation}
where the state $x_k = \begin{bmatrix}x_{1,k} & x_{2,k}\end{bmatrix}^\top \in \mathbb{R}^2$
represents the energy levels in two thermal storage tanks,
$u_k = \begin{bmatrix}u_{1,k} & u_{2,k}\end{bmatrix}^\top \in \mathbb{R}^2$
are continuous control inputs (heat pumps),
$\delta_k \in \{0,1,2,3\}$ is an integer control input (number of active heating rods),
and $d_k = \begin{bmatrix}d_{1,k} & d_{2,k}\end{bmatrix}^\top \in \mathbb{R}^2$
is a known disturbance signal.

The system matrices are
\begin{align}
A &= 
\begin{bmatrix}
\alpha_1 & \nu \\[3pt]
0 & \alpha_2 - \nu
\end{bmatrix},
&
B_u &=
\begin{bmatrix}
b_1 & 0 \\[3pt]
0 & b_2
\end{bmatrix},
\\[6pt]
B_\delta &=
\begin{bmatrix}
0 \\[3pt]
b_3
\end{bmatrix},
&
E &=
\begin{bmatrix}
-b_4 & 0 \\[3pt]
0 & -b_5
\end{bmatrix},
\end{align}
with numerical parameters
$$
\alpha_1 = 0.9983,\quad
\alpha_2 = 0.9966,\quad
\nu = 0.001,\quad
b_1 = b_2 = 0.075,\quad
b_3 = 0.0825,\quad
b_4 = b_5 = 0.0833,
$$
and the sampling period $T = 300~\mathrm{s}$.

The feasible domains are
\begin{align}
0 \le x_{1,k} \le 8.4, \qquad
0 \le x_{2,k} \le 3.6, \\
u_{1,k} \ge 0,\quad u_{2,k} \ge 0,\quad
0 \le u_{1,k} + u_{2,k} \le 8, \\
\delta_k \in \{0,1,2,3\}.
\end{align}

The reference trajectory is constant,
$r = \begin{bmatrix} 4.2 \\ 1.8 \end{bmatrix}$,
and the quadratic stage and terminal cost functions are
\begin{align}
\ell_k(x_k,u_k,\delta_k;r)
  &= \|x_k - r\|_Q^2 + \|u_k\|_R^2 + \|\delta_k\|_\rho^2
     + c_x\,\phi_x(x_k) + c_u\,\phi_u(u_k), \\[3pt]
\ell_N(x_N;r) &= \|x_N - r\|_P^2,
\end{align}
where the weights are
$$
Q = P = \mathrm{diag}(1,1), \quad
R = \mathrm{diag}(0.5,0.5), \quad
\rho = 0.1, \quad
c_x = c_u = 25,
$$
and $\phi_x(\cdot), \phi_u(\cdot)$ are quadratic hinge penalties
that softly enforce the state and input bounds.

The resulting mixed-integer optimal control problem over horizon $N$ is

\begin{align}
\min_{\{x_k,u_k,\delta_k\}_{k=0}^{N-1}}
&\;
\ell_N(x_N;r)
+ \sum_{k=0}^{N-1}
\Big[
  \ell_k(x_k,u_k,\delta_k;r)
\Big] \\
\text{s.t. } \;& 
x_{k+1} = A x_k + B_u u_k + B_\delta \delta_k + E d_k,
\quad k=0,\dots,N-1, \nonumber\\
& 0 \le x_{1,k} \le 8.4,\quad 0 \le x_{2,k} \le 3.6, \nonumber\\
& u_{1,k} \ge 0,\quad u_{2,k} \ge 0,\quad
0 \le u_{1,k} + u_{2,k} \le 8, \nonumber\\
& \delta_k \in \{0,1,2,3\}, \quad x_0~\text{given.} \nonumber
\end{align}

This corresponds to the “\textbf{Numerical Example – Two-Tank Thermal System}”
from \emph{Boldocký et al., 2025}, inspired by the domestic-heating benchmark of
\emph{Löhr et al., 2022}.

## Data Collection 

In [10]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [13]:
def solve_two_tank_miqp(
    N: int,
    x0: np.ndarray,
    d_seq: np.ndarray,
    r: np.ndarray = np.array([4.2, 1.8]),
    Q: np.ndarray = np.diag([1.0, 1.0]),
    R: np.ndarray = np.diag([0.5, 0.5]),
    P: np.ndarray = np.diag([1.0, 1.0]),
    rho: float = 0.1,
    # bounds
    x_lo: np.ndarray = np.array([0.0, 0.0]),
    x_hi: np.ndarray = np.array([8.4, 3.6]),
    u_sum_hi: float = 8.0,
    # penalties (set to None to enforce hard bounds instead)
    c_x: float | None = 25.0,
    c_u: float | None = 25.0,
    time_limit: float | None = None,
    verbose: bool = True,
):
    """
    Parameters
    ----------
    N : horizon length (stages)
    x0 : shape (2,), initial state
    d_seq : shape (N, 2), known disturbances for k=0..N-1
    r : shape (2,), constant reference
    (Q, R, P, rho) : cost weights
    x_lo, x_hi : state bounds
    u_sum_hi : bound for u1+u2
    c_x, c_u : soft penalty weights (quadratic hinge). If None, constraints are hard.
    time_limit : optional Gurobi time limit (seconds)
    """
    assert x0.shape == (2,)
    assert d_seq.shape == (N, 2)
    assert r.shape == (2,)

    # ---- System matrices and sampling (from paper) ----
    alpha1, alpha2, nu = 0.9983, 0.9966, 0.001
    b1 = b2 = 0.075
    b3 = 0.0825
    b4 = b5 = 0.0833
    # T = 300  # seconds (sampling), not directly used in solver

    A = np.array([[alpha1, nu],
                  [0.0,     alpha2 - nu]])
    Bu = np.array([[b1, 0.0],
                   [0.0, b2]])
    Bδ = np.array([[0.0],
                   [b3]])
    E  = np.array([[-b4, 0.0],
                   [0.0, -b5]])

    m = gp.Model("two_tank_miqp")
    if not verbose:
        m.Params.OutputFlag = 0
    if time_limit is not None:
        m.Params.TimeLimit = time_limit

    # ---- Decision variables ----
    # States x[0..N], Inputs u[0..N-1], Integer δ[0..N-1]
    x = m.addVars(N+1, 2, vtype=GRB.CONTINUOUS, name="x")
    u = m.addVars(N,   2, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="u")
    delta = m.addVars(N, vtype=GRB.INTEGER, name="delta")

    # δ ∈ {0,1,2,3}
    for k in range(N):
        m.addConstr(delta[k] >= 0)
        m.addConstr(delta[k] <= 3)

    # Initial condition
    m.addConstr(x[0,0] == float(x0[0]))
    m.addConstr(x[0,1] == float(x0[1]))

    # Dynamics
    for k in range(N):
        # x_{k+1} = A x_k + Bu u_k + Bδ δ_k + E d_k
        Akx  = A @ np.array([x[k,0], x[k,1]])
        Buk  = Bu @ np.array([u[k,0], u[k,1]])
        Bdel = Bδ.flatten() * delta[k]  # broadcast (2,) * scalar-int
        Edk  = E @ d_seq[k]

        # Expand equalities component-wise (Gurobi needs affine combinations)
        # x[k+1, i] == sum_j A[i,j] x[k,j] + sum_j Bu[i,j] u[k,j] + Bδ[i]*delta[k] + sum_j E[i,j]*d[k,j]
        for i in range(2):
            m.addConstr(
                x[k+1, i] ==
                A[i,0]*x[k,0] + A[i,1]*x[k,1] +
                Bu[i,0]*u[k,0] + Bu[i,1]*u[k,1] +
                Bδ[i,0]*delta[k] +
                E[i,0]*d_seq[k,0] + E[i,1]*d_seq[k,1]
            )

    # ---- Bounds: hard or soft ----
    # State bounds
    if c_x is None:  # hard constraints
        for k in range(N+1):
            for i in range(2):
                m.addConstr(x[k,i] >= x_lo[i])
                m.addConstr(x[k,i] <= x_hi[i])
        x_slacks = None
    else:
        # quadratic hinge via nonnegative slacks v_lo >= x_lo - x, v_hi >= x - x_hi
        x_v_lo = m.addVars(N+1, 2, lb=0.0, name="x_violate_lo")
        x_v_hi = m.addVars(N+1, 2, lb=0.0, name="x_violate_hi")
        for k in range(N+1):
            for i in range(2):
                m.addConstr(x_v_lo[k,i] >= x_lo[i] - x[k,i])
                m.addConstr(x_v_hi[k,i] >= x[k,i] - x_hi[i])
        x_slacks = (x_v_lo, x_v_hi)

    # Input lower bounds u >= 0 and sum bound u1+u2 <= u_sum_hi
    if c_u is None:  # hard constraints
        for k in range(N):
            m.addConstr(u[k,0] >= 0.0)
            m.addConstr(u[k,1] >= 0.0)
            m.addConstr(u[k,0] + u[k,1] <= u_sum_hi)
        u_slacks = None
    else:
        # Violations: v_u_lo1 >= -u1, v_u_lo2 >= -u2, v_usum >= u1+u2 - u_sum_hi
        v_u_lo1 = m.addVars(N, lb=0.0, name="u1_violate_lo")
        v_u_lo2 = m.addVars(N, lb=0.0, name="u2_violate_lo")
        v_usum  = m.addVars(N, lb=0.0, name="usum_violate_hi")
        for k in range(N):
            m.addConstr(v_u_lo1[k] >= -u[k,0])
            m.addConstr(v_u_lo2[k] >= -u[k,1])
            m.addConstr(v_usum[k]  >=  u[k,0] + u[k,1] - u_sum_hi)
        u_slacks = (v_u_lo1, v_u_lo2, v_usum)

    # ---- Objective ----
    obj = gp.QuadExpr(0.0)

    # Stage costs
    for k in range(N):
        # (x_k - r)' Q (x_k - r)
        for i in range(2):
            for j in range(2):
                obj += Q[i,j] * (x[k,i] - float(r[i])) * (x[k,j] - float(r[j]))
        # u_k' R u_k
        for i in range(2):
            for j in range(2):
                obj += R[i,j] * u[k,i] * u[k,j]
        # rho * delta_k^2
        obj += float(rho) * delta[k] * delta[k]

        # soft penalties, if enabled
        if x_slacks is not None:
            x_v_lo, x_v_hi = x_slacks
            for i in range(2):
                obj += float(c_x) * x_v_lo[k,i] * x_v_lo[k,i]
                obj += float(c_x) * x_v_hi[k,i] * x_v_hi[k,i]
        if u_slacks is not None:
            v_u_lo1, v_u_lo2, v_usum = u_slacks
            obj += float(c_u) * v_u_lo1[k] * v_u_lo1[k]
            obj += float(c_u) * v_u_lo2[k] * v_u_lo2[k]
            obj += float(c_u) * v_usum[k]  * v_usum[k]

    # Terminal cost at k = N: (x_N - r)' P (x_N - r)
    for i in range(2):
        for j in range(2):
            obj += P[i,j] * (x[N,i] - float(r[i])) * (x[N,j] - float(r[j]))

    m.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    m.optimize()

    # ---- Extract solution ----
    status = m.Status
    sol = {
        "status": status,
        "obj_val": m.ObjVal if status == GRB.OPTIMAL or status == GRB.TIME_LIMIT else None,
        "x": np.array([[x[k,i].X for i in range(2)] for k in range(N+1)]) if m.SolCount > 0 else None,
        "u": np.array([[u[k,i].X for i in range(2)] for k in range(N)]) if m.SolCount > 0 else None,
        "delta": np.array([int(round(delta[k].X)) for k in range(N)]) if m.SolCount > 0 else None,
    }
    return sol

In [14]:
N = 24  # 24 steps (~2 hours at 300s sampling)
x0 = np.array([2.0, 1.0])
# Disturbances d_k; here a simple demo pattern. Replace with your data.
d_seq = np.zeros((N, 2))
d_seq[:,0] = 2.0  # baseline consumption from tank 1
d_seq[8:12,1] = 6.0  # a burst on tank 2

# Solve with soft penalties (like the paper)
sol = solve_two_tank_miqp(
        N=N, x0=x0, d_seq=d_seq,
        c_x=25.0, c_u=25.0,  # soft constraints
        time_limit=10, verbose=True
)
# print("Status:", sol["status"])
print("Objective:", sol["obj_val"])
print("delta:", sol["delta"])
print("u[0]:", sol["u"][0], "x[0]:", sol["x"][0], "x[N]:", sol["x"][-1])

Set parameter TimeLimit to value 10
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i9-14900F, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  10

Optimize a model with 270 rows, 294 columns and 610 nonzeros
Model fingerprint: 0xb22d9727
Model has 290 quadratic objective terms
Variable types: 270 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [4e+00, 8e+00]
  QObjective range [2e-01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 8e+00]
Found heuristic solution: objective 1680909.9498
Presolve removed 104 rows and 56 columns
Presolve time: 0.00s
Presolved: 166 rows, 238 columns, 449 nonzeros
Presolved model has 238 quadratic objective terms


Found heuristic solution: objective 1180909.9498
Variable types: 214 continuous, 24 integer (0 binary)

Root relaxation: objective 1.519321e+02, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  151.93209    0   24 1180909.95  151.93209   100%     -    0s
H    0     0                     152.5864734  151.93209  0.43%     -    0s
H    0     0                     152.2493932  151.93209  0.21%     -    0s
     0     0  151.96417    0   24  152.24939  151.96417  0.19%     -    0s
     0     2  151.96417    0   24  152.24939  151.96417  0.19%     -    0s
H    5     8                     152.2331993  151.99333  0.16%   1.6    0s
H    6     8                     152.2272458  151.99462  0.15%   1.7    0s

Explored 2206 nodes (4487 simplex iterations) in 0.05 seconds (0.03 work units)
Thread count was 32 (of 32 available processors)

S

In [ ]:
Objective: 152.22724582143286
delta: [2 2 2 2 1 1 2 2 2 3 2 2 2 1 1 1 1 0 0 0 0 0 0 0]
u[0]: [4.79681432 0.50044853] x[0]: [2. 1.] x[N]: [2.36068082 1.68389984]